In [1]:
with open('Day16.txt') as f:
    data = f.read()

In [2]:
import re
regex_field = re.compile(r'([\w\s]+): (\d+)-(\d+) or (\d+)-(\d+)')

fields, my_ticket, tickets = {}, [], []
for part, block in enumerate(data.split('\n\n')):
    if part == 0:  # ticket fields
        for line in block.splitlines():
            field_name, mina, maxa, minb, maxb = regex_field.match(line).groups()
            mina, maxa, minb, maxb = int(mina), int(maxa), int(minb), int(maxb)
            fields[field_name] = (
                lambda value, mina=mina, maxa=maxa, minb=minb, maxb=maxb: (mina <= value <= maxa) or (minb <= value <= maxb))
    elif part == 1:  # my ticket
        my_ticket = list(map(int, block.replace('your ticket:\n', '').split(',')))
    elif part == 2:  # others tickets
        for index, line in enumerate(block.splitlines()):
            if not index:
                continue
            tickets.append(list(map(int, line.split(','))))

In [3]:
def get_valid_tickets(tickets, fields):
    valid_tickets = []
    error_rate = 0
    for ticket in tickets:
        is_valid = True
        for value in ticket:
            for field_name, check in fields.items():
                if check(value):
                    break
            else:
                is_valid = False
                error_rate += value
        if is_valid:
            valid_tickets.append(ticket)
    return valid_tickets, error_rate

In [4]:
%time valid_tickets, error_rate = get_valid_tickets(tickets, fields)
error_rate

CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 1.62 ms


23044

In [5]:
def find_valid_fields(tickets, fields):
    valid_fields = {}
    for index in range(len(my_ticket)):
        checks = set(fields.keys())
        for value in (ticket[index] for ticket in tickets):
            for name, check in fields.items():
                if name not in checks:
                    continue
                if not check(value):
                    checks.remove(name)
        valid_fields[index] = checks
    used_fields = set()
    for index, field_set in sorted(valid_fields.items(), key=lambda e: len(e[1])):
        valid_fields[index] = next(iter(field_set - used_fields))
        used_fields.update(field_set)
    return valid_fields

In [6]:
%time valid_fields = find_valid_fields(valid_tickets, fields)

CPU times: user 12.4 ms, sys: 0 ns, total: 12.4 ms
Wall time: 12.3 ms


In [7]:
from functools import reduce
from operator import mul

reduce(mul, (my_ticket[index] for index, field_name in valid_fields.items() if 'departure' in field_name))

3765150732757